# Soft Singular Value Thresholing for Image Inpaintgs

### Imports and Helper Functions

This section imports the required Python libraries and defines two helper functions:

- **`gen_mask`**: Generates a random boolean mask simulating missing pixels for inpainting.
- **`resize_image`**: Resizes a PIL image while preserving aspect ratio.

**References:**
- Cai, Candès, and Shen (2008). *A Singular Value Thresholding Algorithm for Matrix Completion*. [arXiv:0810.3286](https://arxiv.org/pdf/0810.3286)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


# Generate a random boolean mask to simulate missing pixels in an image
def gen_mask(A, pct, random_state=0):
    shape = A.shape
    ndims = np.prod(shape, dtype=np.int32)
    mask = np.full(ndims, False)
    mask[: int(pct * ndims)] = True
    np.random.seed(random_state)
    np.random.shuffle(mask)
    mask = mask.reshape(shape)
    return mask

# Resize an image to a specified size while preserving aspect ratio
def resize_image(img, size):
    img = img.copy()
    img.thumbnail(size=(size, size))
    return img

### Image Loading and Preprocessing

1. Load an image from the specified path.
2. Resize it to a smaller width for computational efficiency.
3. Convert the image to grayscale and normalize pixel values to [0,1].
4. Display the processed grayscale image.

This prepares the data for the inpainting algorithm.

In [ ]:
fname = "./landscape.jpg"
fname = "../LAB01/mondrian.jpg"
raw_img = Image.open(fname)
size = 400
img = resize_image(raw_img, size)
print(f"Raw image size: {raw_img.size}")
print(f"Resized image size: {img.size}")

img = np.array(img) / 255
img = img.mean(axis=-1)
plt.imshow(img, cmap="gray")

Implements the **SVT algorithm** for image inpainting
1. Generate a random mask to simulate missing pixels (using `gen_mask`).
2. Initialize parameters and variables for the SVT algorithm.
3. Iteratively perform singular value thresholding and update estimates.
4. Visualize intermediate reconstructions and convergence behavior.

In [ ]:
# Inpainting percentage
pct = 0.5
mask = gen_mask(A=img, pct=pct, random_state=0)

# parameters
n_iters = 700
tol = 0.01

delta = 1.2
tau = 5 * np.sqrt(img.shape[0]*img.shape[1])
c0 = np.ceil(tau / (delta * np.linalg.norm(img, ord=2)))

X = img.copy()
M = np.zeros_like(X)
Y = c0 * delta * mask * X

prev_err = np.inf
best_X = M.copy()

for i in range(n_iters):
    u, s, vh = np.linalg.svd(Y, full_matrices=False)
    # soft thresholding
    shrink_s = np.maximum(s - tau, 0)
    # reconstruct X
    M = (u * shrink_s) @ vh
    # update Y
    Y += delta * mask * (X - M)
    # check the error
    abs_err = np.linalg.norm(mask * (M - X), ord="fro")
    err = abs_err / np.linalg.norm(mask * X, ord="fro")
    print(f"Iteration: {i}; err: {err:.6f}")

    # save best
    if err < prev_err:
        prev_err = err
        best_X = np.clip(M, a_min=0, a_max=1)

    # visualize
    if i % 40 == 0:
        fig, axs = plt.subplots(2, 2, figsize=(15, 7))
        axs[0, 0].set_title("Original Image")
        axs[0, 0].imshow(X, cmap="gray")
        axs[0, 0].axis("off")
        axs[0, 1].set_title("Dirty Image")
        axs[0, 1].imshow(X * mask, cmap="gray")
        axs[0, 1].axis("off")
        axs[1, 0].set_title("Reconstructed Image")
        axs[1, 0].imshow(best_X, cmap="gray")
        axs[1, 0].axis("off")
        axs[1, 1].set_title("Error")
        axs[1, 1].imshow(np.abs(best_X - img), vmin=0, vmax=1, cmap="gray")
        axs[1, 1].axis("off")
        plt.show()

    if err <= tol:
        break

### Homework
Apply the algorithm to each channel of the image and obtain a reconstructed color image